# HLS
HLSをやってみよう！

- Pynq_test_hls

In [ ]:
from pynq import Overlay
OL = Overlay("/home/xilinx/pynq/overlays/my_design/top_design_wrapper.bit")
OL.download()

## show IPs
print(OL.ip_dict.keys())

## GPIO
とりあえずLチカするのが礼儀。

In [ ]:
from time import sleep
## GPIO
OL.axi_gpio_1.register_map.GPIO_TRI = 0 ## Output
OL.axi_gpio_1.register_map.GPIO_DATA = 0xF

lst = [
    0x7, 0x3, 0x1, 0x0,
    0x1, 0x2, 0x4, 0x8,
    0xF, 0x0, 0xF, 0x0,
    0xF, 0x0, 0xF, 0x0]

for l in lst:
    sleep(0.2)
    OL.axi_gpio_1.register_map.GPIO_DATA = l


## IP
自作HLS IP(pynq_hls/sample_func.cpp)を動かしてみる。

0. GPIOのレジスタのアドレスをIPに設定
0. IPをスタートする
  - IP内部フロー
      - GPIO[0]がsample_in
      - GPIO2[31:0]を外部アドレスextMemPtr
      - アドレスからリード
      - データ処理(32bit反転)
      - 結果をレジスタにセット
0. IPレジスタから読み出す。

In [ ]:
OL.axi_gpio_0.register_map.GPIO2_DATA = 0x000000FF

adr = OL.ip_dict['axi_gpio_0']['phys_addr']
offset = OL.axi_gpio_0.register_map.GPIO2_DATA.address
data = OL.axi_gpio_0.register_map.GPIO2_DATA
print(f"Address: ", hex(adr), hex(offset), hex(data))

OL.pynq_sample_func_0.register_map.baseMemAdr_V = adr + offset
## OL.pynq_sample_func_0.register_map.CTRL.AUTO_RESTART = 1
OL.pynq_sample_func_0.register_map.CTRL.AP_START = 1
sample_data = OL.pynq_sample_func_0.register_map.sample_data0

print(f"SampleData: {sample_data}")

### sample_in, Mode変更

In [ ]:
OL.axi_gpio_0.register_map.GPIO_DATA = 0x1 ## Mode 1 as 2bit Shift

OL.pynq_sample_func_0.register_map.CTRL.AP_START = 1

sample_data = OL.pynq_sample_func_0.register_map.sample_data0

OL.axi_gpio_0.register_map.GPIO_DATA = 0x0 ## Mode 0 as Invert

print(f"SampleData: {sample_data}")

### Zynq通して、DDR

In [ ]:
OL.processing_system7_0.cma_mem_size()/1024/1024

In [ ]:
buf = OL.processing_system7_0.cma_array([1024,1024])
## OL.processing_system7_0.cma_get_phy_addr(
hex(buf.physical_address)

In [ ]:
buf[0][0] = 0xFF

OL.pynq_sample_func_0.register_map.baseMemAdr_V = buf.physical_address
OL.pynq_sample_func_0.register_map.CTRL.AP_START = 1
sample_data = OL.pynq_sample_func_0.register_map.sample_data0

print(f"Addr: {OL.pynq_sample_func_0.register_map.baseMemAdr_V}")
print(f"SampleData: {sample_data}")